In [1]:
!pip install scikit-learn-intelex --progress-bar off >> /tmp/pip_sklearnex.log

In [2]:
import pandas as pd
import numpy as np
from sklearnex import patch_sklearn
patch_sklearn()
from sklearn import *
import glob

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
p = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/'

train = glob.glob(p+'train/**/**')
test = glob.glob(p+'test/**/**')
subjects = pd.read_csv(p+'subjects.csv')
tasks = pd.read_csv(p+'tasks.csv')
sub = pd.read_csv(p+'sample_submission.csv')

In [4]:
tasks['Duration'] = tasks['End'] - tasks['Begin']
tasks = pd.pivot_table(tasks, values=['Duration'], index=['Id'], columns=['Task'], aggfunc='sum', fill_value=0)
tasks.columns = [c[-1] for c in tasks.columns]
tasks = tasks.reset_index()
tasks['t_kmeans'] = cluster.KMeans(n_clusters=10, random_state=3).fit_predict(tasks[tasks.columns[1:]])

subjects = subjects.fillna(0).groupby('Subject').median()
subjects = subjects.reset_index()
subjects.rename(columns={'Subject':'Id'}, inplace=True)
subjects['s_kmeans'] = cluster.KMeans(n_clusters=10, random_state=3).fit_predict(subjects[subjects.columns[1:]])

In [5]:
def reader(f):
    try:
        df = pd.read_csv(f, usecols=['Time', 'AccV', 'AccML', 'AccAP', 'StartHesitation', 'Turn' , 'Walking'])
        df['Id'] = f.split('/')[-1].split('.')[0]
        df['data_type'] = f.split('/')[-2]
        df = df.fillna(99)
        return df
    except: pass
train = pd.concat([reader(f) for f in train]).fillna(0); print(train.shape)
train['data_type'] = np.where(train['data_type'] != 'defog', 0, 1)
cols = [c for c in train.columns if c not in ['Id', 'StartHesitation', 'Turn' , 'Walking', 'Valid', 'Task','Event']]
pcols = ['StartHesitation', 'Turn' , 'Walking']
scols = ['Id', 'StartHesitation', 'Turn' , 'Walking']

(20588374, 9)


In [6]:
train

,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,Id,data_type
0,0,-1.002697,0.022371,0.068304,0,0,0,be9d33541d,1
1,1,-1.002641,0.019173,0.066162,0,0,0,be9d33541d,1
2,2,-0.999820,0.019142,0.067536,0,0,0,be9d33541d,1
3,3,-0.998023,0.018378,0.068409,0,0,0,be9d33541d,1
4,4,-0.998359,0.016726,0.066448,0,0,0,be9d33541d,1
...,...,...,...,...,...,...,...,...,...
5153,5153,-9.915920,-0.105897,-1.123455,0,0,0,0506d9a39f,0
5154,5154,-9.693752,-0.066892,-1.114903,0,0,0,0506d9a39f,0
5155,5155,-9.548118,-0.098315,-1.112123,0,0,0,0506d9a39f,0
5156,5156,-9.469803,-0.111004,-1.130814,0,0,0,0506d9a39f,0


In [7]:
cols

['Time', 'AccV', 'AccML', 'AccAP', 'data_type']

In [8]:
x1, x2, y1, y2 = model_selection.train_test_split(train[cols], train[pcols], test_size=.30, random_state=3, stratify=train[pcols])
del train
reg = ensemble.ExtraTreesRegressor(n_estimators=100, max_depth=4, max_features = 'sqrt', n_jobs=-1, random_state=3)
reg.fit(x1[:5_000_000],y1[:5_000_000])
print(metrics.average_precision_score(y2, reg.predict(x2).clip(0.0,1.0)))

0.2651249814337529


In [9]:
sub['t'] = 0
submission = []
for f in test:
    df = pd.read_csv(f)
    df['Id'] = f.split('/')[-1].split('.')[0]
    df['data_type'] = f.split('/')[-2]
    df['data_type'] = np.where(df['data_type'] != 'defog', 0, 1)       
    df = df.fillna(0).reset_index(drop=True)
    df = df.fillna(99)
    res = pd.DataFrame(reg.predict(df[cols]).clip(0.0,1.0), columns=pcols)
    df = pd.concat([df,res], axis=1)
    df['Id'] = df['Id'].astype(str) + '_' + df['Time'].astype(str)
    submission.append(df[scols])
submission = pd.concat(submission)
submission = pd.merge(sub[['Id','t']], submission, how='left', on='Id').fillna(0.0)
submission[scols].to_csv('submission.csv', index=False)